# Analyze data using Orbit

This notebook shows how to analyze images. A model has been generated and uploaded to the server.

In [2]:
%classpath add jar /opt/java-apps/orbit/lib/*

In [3]:
g = new EasyForm("Enter credentials to connect to OMERO")
g.addTextField("Server").onInit({g['Server'] = "outreach.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g

In [4]:
ga = new EasyForm("Select the Image to analyze")
ga.addTextField("ImageID")
ga

In [ ]:
import com.actelion.research.orbit.beans.RawDataFile
import com.actelion.research.orbit.beans.RawAnnotation
import com.actelion.research.orbit.imageAnalysis.dal.DALConfig
import com.actelion.research.orbit.imageAnalysis.models.OrbitModel
import com.actelion.research.orbit.imageAnalysis.models.SegmentationResult
import com.actelion.research.orbit.imageAnalysis.utils.OrbitHelper

import java.awt.Shape

import com.actelion.research.orbit.imageAnalysis.components.*
import com.actelion.research.orbit.imageAnalysis.models.*
import com.actelion.research.orbit.imageprovider.ImageProviderOmero
import com.actelion.research.orbit.imageprovider.OmeroConf
import omero.gateway.model.*
import omero.model.*
import omero.gateway.Gateway
import omero.gateway.SecurityContext
import static omero.rtypes.rstring
import static omero.rtypes.rint
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility

// Edit these parameters
String hostName = g['Server']
String userName = g['UserName']
String password = g['Password']

// Get the OMERO Image ID
long omeroImageId = ga['ImageID'].toLong()

// Login to create a new connection with OMERO
ImageProviderOmero imageProvider = new ImageProviderOmero(new OmeroConf(hostName,4064,true))

imageProvider.authenticateUser(userName, password)
println("Connected ")

Gateway gateway = imageProvider.getGatewayAndCtx().getGateway()
SecurityContext ctx = imageProvider.getGatewayAndCtx().getCtx()

// Load Models that I own. OMERO annotations of type: Model
imageProvider.setOnlyOwnerObjects(true)
List<RawAnnotation> annotations = imageProvider.LoadRawAnnotationsByType(RawAnnotation.ANNOTATION_TYPE_MODEL)
println("Found " + annotations.size() + " files")
// Use the first annotation
int fileAnnId = annotations[0].getRawAnnotationId()
OrbitModel model = OrbitModel.LoadFromOrbit(fileAnnId)
println("Loaded Model: " + model.getName())

// Perform the segmentation
SegmentationResult res = OrbitHelper.Segmentation(rdf.rawDataFileId, model, null, 1)

// handle the segmented objects
println("SegmentationResult: " + res.shapeList.size() + " shapes")
for (Shape shape: res.shapeList) {
    // can cast shape to Polygon or simply listPoints
    String points = shape.listPoints()

    // Create Polygon in OMERO
    p = new PolygonI()
    // Convert "x, y; x, y" format to "x, y, x, y" for OMERO
    points = points.replace(";", ",")
    p.setPoints(rstring(points))
    p.setTheT(rint(0))
    p.setTheZ(rint(0))
    p.setStrokeColor(rint(-65281))   // yellow

    // Add each shape to an ROI on the Image
    ImageI image = new ImageI(omeroImageId, false)
    RoiI roi = new RoiI()
    roi.setImage(image)
    roi.addShape(p)

    // Save
    gateway.getUpdateService(ctx).saveAndReturnObject(roi)
}

println("Close...")
imageProvider.close()


Connected 
Found 1 files
